In [8]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, roc_auc_score, cohen_kappa_score
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.feature_selection import VarianceThreshold
from sklearn.utils import shuffle

# Load the dataset
df = pd.read_csv("C:/RHEA/S7/Project/Code/Datasets/Lung_gene.csv", index_col="Unnamed: 0")

print(df.columns)

# Encode the target variable
le = LabelEncoder()
df["classes"] = le.fit_transform(df["classes"])  # Convert categorical labels to 0 and 1

# Define features and target
X = df.drop(columns=["classes"])
y = df["classes"]

# Split dataset into training and testing sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)


df = df.iloc[:,1:]
     

df.dropna()

Index(['TSPAN6', 'TNMD', 'DPM1', 'SCYL3', 'FIRRM', 'FGR', 'CFH', 'FUCA2',
       'GCLC', 'NFYA',
       ...
       'SNORA74C-2', 'ELOA3BP', 'NPBWR1', 'ELOA3DP', 'PDCD6-AHRR', 'LNCDAT',
       'LOC124902537', 'RNF228', 'PANO1', 'classes'],
      dtype='object', length=31506)


,TNMD,DPM1,SCYL3,FIRRM,FGR,CFH,FUCA2,GCLC,NFYA,STPG1,...,SNORA74C-2,ELOA3BP,NPBWR1,ELOA3DP,PDCD6-AHRR,LNCDAT,LOC124902537,RNF228,PANO1,classes
TCGA-60-2712-01A-01R-0851-07,0.000000,10.835261,9.623881,9.335390,10.839204,12.055282,10.528454,12.670656,10.584963,8.861087,...,0.0,0.0,3.169925,0.0,0.0,0.000000,0.000000,6.339850,4.643856,0
TCGA-56-7221-01A-11R-2045-07,3.321928,11.707790,9.743151,10.067434,9.082149,11.060696,11.078818,15.292573,11.278449,10.490851,...,0.0,0.0,9.167418,0.0,0.0,0.000000,0.000000,8.717676,4.700440,0
TCGA-21-A5DI-01A-31R-A26W-07,0.000000,11.122828,8.174926,8.049849,7.339850,11.634357,9.807355,12.055282,10.453271,9.577429,...,0.0,0.0,7.851749,0.0,1.0,0.000000,0.000000,6.686501,3.459432,0
TCGA-43-7657-11A-01R-2125-07,0.000000,10.760720,9.636625,7.693487,11.921097,13.641262,11.542065,10.691744,10.557464,10.151017,...,0.0,0.0,3.459432,0.0,0.0,0.000000,3.321928,4.392317,3.321928,1
TCGA-43-7657-01A-31R-2125-07,0.000000,11.307201,8.957102,8.764872,8.751544,10.285402,11.861087,13.788311,10.868823,10.598983,...,0.0,0.0,13.429014,0.0,0.0,2.584963,0.000000,6.954196,4.087463,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-39-5028-01A-01R-1443-07,1.584963,11.633903,9.194757,8.294621,9.419960,13.058499,11.092757,11.814182,10.359750,10.213104,...,0.0,0.0,4.087463,0.0,0.0,1.000000,0.000000,4.000000,5.129283,0
TCGA-NC-A5HE-01A-11R-A26W-07,1.584963,11.428884,10.029287,9.897845,11.770251,11.486835,11.574594,13.891689,11.237807,8.159871,...,0.0,0.0,6.339850,0.0,1.0,3.000000,0.000000,5.584963,5.357552,0
TCGA-66-2783-01A-01R-1201-07,0.000000,10.348728,9.262095,9.189825,9.033423,10.631177,10.101976,12.842743,10.290019,10.030667,...,0.0,0.0,6.768184,0.0,0.0,0.000000,0.000000,6.475733,4.000000,0
TCGA-66-2795-01A-02R-0980-07,0.000000,11.051209,9.601771,9.731319,9.019591,11.106563,10.083479,14.560751,10.695228,8.375039,...,0.0,0.0,7.139551,0.0,0.0,1.584963,0.000000,5.807355,2.321928,0


In [9]:
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)

# 4. Get feature importances

feature_importances = model.feature_importances_
feature_names = X.columns

# Create a DataFrame for importances
importance_df = pd.DataFrame({
    'Gene': feature_names,
    'Importance': feature_importances
})

# ---------------------------
# 5. Select Top 1000 Genes
# ---------------------------
top_genes_df = importance_df.sort_values(by='Importance', ascending=False).head(1000)




c:\RHEA\S7\Project\Code\Cancer-Classification\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [20:07:11] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [10]:
# Train XGBoost to get feature importances
xgb_selector = XGBClassifier(n_estimators=500, learning_rate=0.05, max_depth=6,
                             subsample=0.8, colsample_bytree=0.8, random_state=42,
                             use_label_encoder=False, eval_metric="logloss")

xgb_selector.fit(X_train, y_train)

# Get feature importances
feature_importances = pd.Series(xgb_selector.feature_importances_, index=X.columns)

# List of feature selection counts
feature_counts = [10, 50, 100, 500, 1000]

# Initialize a DataFrame to store results
results_df = pd.DataFrame(columns=["Features", "AUC", "Accuracy", "Precision", "Kappa"])

# Loop through different feature counts
for num_features in feature_counts:
    # Select top N features
    top_features = feature_importances.nlargest(num_features).index.tolist()

    # Filter dataset with selected features
    X_train_selected = X_train[top_features]
    X_test_selected = X_test[top_features]

    # Train Random Forest classifier
    rf_clf = RandomForestClassifier(n_estimators=500, max_depth=None, min_samples_split=2,
                                    random_state=42, n_jobs=-1)
    rf_clf.fit(X_train_selected, y_train)

    # Make predictions
    y_pred = rf_clf.predict(X_test_selected)
    y_pred_proba = rf_clf.predict_proba(X_test_selected)[:, 1]  # Probability estimates for AUC calculation

    # Evaluate model performance
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred_proba)
    kappa = cohen_kappa_score(y_test, y_pred)

    # Store results in DataFrame
    results_df = pd.concat([results_df, pd.DataFrame([[num_features, auc, accuracy, precision, kappa]],
                                                      columns=["Features", "AUC", "Accuracy", "Precision", "Kappa"])],
                                                      ignore_index=True)

# Display final results
print(results_df)
     

c:\RHEA\S7\Project\Code\Cancer-Classification\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [20:07:43] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\rheam\AppData\Local\Temp\ipykernel_21888\1108089128.py:42: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, pd.DataFrame([[num_features, auc, accuracy, precision, kappa]],


  Features       AUC  Accuracy  Precision     Kappa
0       10  0.994805  0.982249   0.928571  0.886856
1       50  0.994372  0.982249   0.928571  0.886856
2      100  0.994805  0.982249   0.928571  0.886856
3      500  0.995238  0.982249   0.928571  0.886856
4     1000  0.995238  0.988166   0.933333  0.926840
